In [1]:
import pandas as pd
import numpy as np
import time
pd.set_option('mode.chained_assignment', None) # turn off warning message of SettingWithCopyWarning 

In [2]:
df = pd.read_csv('../data/2012-18_playerBoxScore.csv') # reading csv in this way for now.

# Data Wrangling / Feature Engineering  
> * Please refer to **'../doc/metadata_playerBoxScore.pdf'** for description on the meaning of features.  
> * The wrangling part will take about 5 minutes to run, and the result dataframe will be about 140MB.

In [3]:
df.shape

(155713, 52)

In [4]:
df = df.dropna()

In [5]:
df.shape

(155672, 52)

> Only a few observations are dropped by `dropna`.

In [6]:
# filter columns for modeling
cols_to_use = ['gmDate', 'teamAbbr',  'teamLoc', 'teamRslt', 'playDispNm', 'playStat', 'playMin', 'playPos', 'playPTS', 'playAST', 'playTO', 'playSTL', 'playBLK', 'playTRB']
df = df[cols_to_use].copy()

In [7]:
# format datetime for sorting
df['gmDate'] = pd.to_datetime(df['gmDate']).copy()

In [8]:
# replace categorical values with numbers to apply `rolling` to them
rep_dict = {'teamLoc': {'Home':1, 'Away':0},
            'teamRslt': {'Win':1, 'Loss':0},
            'playStat': {'Starter':1, 'Bench':0}}

for x in rep_dict.keys():
    df[x] = df[x].apply(lambda y: rep_dict[x][y])

In [9]:
df_org = df.copy()

> * The following chunk is the main part of the wrangling /feature engineering.  
> * The main idea is to create rolling features as explanatory variables. For instance, to predict the court minutes of a player in the coming game, I might want to know `how many minutes the player played on average in the past 5 games`. This would be one of the rolling features generated below  - `playMin_last5_mean`. And there are many more of them created through iteration. 

In [10]:
df = pd.DataFrame() 

cols_keep = ['playDispNm', 'gmDate', 'teamAbbr', 'playMin', 'teamLoc', 'playStat', 'playPos']
cols_roll = ['teamLoc', 'teamRslt', 'playMin', 'playPTS', 'playAST', 'playTO', 'playSTL', 'playBLK', 'playTRB']
windows = [1, 5, 10, 15, 20]
agg_funcs = ['mean', 'std', 'median']

# iterate through names to make new df with windows features
name_list = list(df_org['playDispNm'].unique())

for name in name_list:
    thisguy = df_org.query("playDispNm == @name").sort_values('gmDate', ascending=True)
    if len(thisguy) < 30: # ignoring players that have less than 30 games' record
        continue
    cols_created = []

    for col in cols_roll:
        for t in windows:
            for fun in agg_funcs:
                if (t == 1) & (fun in ['std', 'median']):
                    continue  # 1-day std and median are useless
                new_col = col+'_last'+str(t)+'_'+fun
                cols_created.append(new_col)
                thisguy.loc[:, new_col] = getattr(thisguy[col].rolling(t, min_periods=1), fun)().copy()

    # shift created features by 1 row so that it means the "last n games"          
    cols_created.append('gmDate')
    merge_temp = thisguy[cols_created].copy().set_index('gmDate').shift(1, axis = 0).reset_index().copy()
    thisguy_result = pd.merge(thisguy[cols_keep], merge_temp, how='left', on='gmDate')

    # append this guy's result table into df
    df = pd.concat((df, thisguy_result), axis=0, ignore_index=True).copy()

In [11]:
df.shape

(152929, 124)

In [12]:
df = df.dropna().copy()

In [13]:
df.shape

(151493, 124)

> Only a few observations are dropped by `dropna`. (should roughly be 1 observation per player)

In [14]:
# wrangling part ends, save the result dataframe
# df.to_csv('../data/player_data_ready.csv', index=False)

#### Some notes on the wrangling part: 
> * There's only "Regular" data. No "pre season" / "post season".  
> * Didn't assign any value to the `win_types` argument of `rolling` function, meaning all rolling values are calculated with equal weights.   
> * Players having less than 30 games' record are filtered out.

# EDA

In [15]:
# import packages again because wrangling part will be a separate script later
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import altair as alt
from IPython.display import display
pd.options.display.max_columns = None # such that jupyter prints out all columns and rows
pd.options.display.max_rows = None

In [16]:
# load data file here

In [17]:
# make primary key column
df['pk'] = df["playDispNm"] + '_' + df["gmDate"].apply(lambda x: x.strftime('%Y-%m-%d')) 

#### Remove Unnecessary Columns

In [19]:
info_cols = ['pk', 'playDispNm', 'gmDate', 'teamAbbr']
df_info = df[info_cols]

df = df.drop(columns=info_cols)

#### Train/test split

In [25]:
df_train, df_test = train_test_split(df, test_size=0.2)

#### Start basic checks

In [21]:
df_train.describe()

,playMin,teamLoc,playStat,teamLoc_last1_mean,teamLoc_last5_mean,teamLoc_last5_std,teamLoc_last5_median,teamLoc_last10_mean,teamLoc_last10_std,teamLoc_last10_median,teamLoc_last15_mean,teamLoc_last15_std,teamLoc_last15_median,teamLoc_last20_mean,teamLoc_last20_std,teamLoc_last20_median,teamRslt_last1_mean,teamRslt_last5_mean,teamRslt_last5_std,teamRslt_last5_median,teamRslt_last10_mean,teamRslt_last10_std,teamRslt_last10_median,teamRslt_last15_mean,teamRslt_last15_std,teamRslt_last15_median,teamRslt_last20_mean,teamRslt_last20_std,teamRslt_last20_median,playMin_last1_mean,playMin_last5_mean,playMin_last5_std,playMin_last5_median,playMin_last10_mean,playMin_last10_std,playMin_last10_median,playMin_last15_mean,playMin_last15_std,playMin_last15_median,playMin_last20_mean,playMin_last20_std,playMin_last20_median,playPTS_last1_mean,playPTS_last5_mean,playPTS_last5_std,playPTS_last5_median,playPTS_last10_mean,playPTS_last10_std,playPTS_last10_median,playPTS_last15_mean,playPTS_last15_std,playPTS_last15_median,playPTS_last20_mean,playPTS_last20_std,playPTS_last20_median,playAST_last1_mean,playAST_last5_mean,playAST_last5_std,playAST_last5_median,playAST_last10_mean,playAST_last10_std,playAST_last10_median,playAST_last15_mean,playAST_last15_std,playAST_last15_median,playAST_last20_mean,playAST_last20_std,playAST_last20_median,playTO_last1_mean,playTO_last5_mean,playTO_last5_std,playTO_last5_median,playTO_last10_mean,playTO_last10_std,playTO_last10_median,playTO_last15_mean,playTO_last15_std,playTO_last15_median,playTO_last20_mean,playTO_last20_std,playTO_last20_median,playSTL_last1_mean,playSTL_last5_mean,playSTL_last5_std,playSTL_last5_median,playSTL_last10_mean,playSTL_last10_std,playSTL_last10_median,playSTL_last15_mean,playSTL_last15_std,playSTL_last15_median,playSTL_last20_mean,playSTL_last20_std,playSTL_last20_median,playBLK_last1_mean,playBLK_last5_mean,playBLK_last5_std,playBLK_last5_median,playBLK_last10_mean,playBLK_last10_std,playBLK_last10_median,playBLK_last15_mean,playBLK_last15_std,playBLK_last15_median,playBLK_last20_mean,playBLK_last20_std,playBLK_last20_median,playTRB_last1_mean,playTRB_last5_mean,playTRB_last5_std,playTRB_last5_median,playTRB_last10_mean,playTRB_last10_std,playTRB_last10_median,playTRB_last15_mean,playTRB_last15_std,playTRB_last15_median,playTRB_last20_mean,playTRB_last20_std,playTRB_last20_median
count,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.00000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,1.211940e+05,121194.000000,121194.000000,121194.000000,121194.00000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.000000,121194.

> * The target feature `playMin` is showing a reasonable statistical summary.  
> * Intuitively important explanatory features like `playMin_last"n"`, `playStat`, `playPTS_last"n"` are showing reasonable statistical summaries as well.

In [22]:
# check missing values
assert df_train.isna().sum().sum() == 0, "There shouldn't be any missing values!"

> There's no missing value in the data (because I've dropped them in the wrangling step).

#### Get the Correlation between all explanatory variables and target

In [23]:
correlations = {}
for col in df:
    if col == 'playMin':
        continue
    try:
        correlations[col] = round(np.corrcoef(df[col], df['playMin'])[0][1], 3)
    except:
        continue
        
correl_df = pd.DataFrame.from_dict(correlations, orient='index')
correl_df.columns = ['corr w/ target']
correl_df = correl_df.sort_values('corr w/ target', ascending=False)
print("""Top pos/neg correlated features against the target: \n{}\n\n{}\n""".format(correl_df.head(20), correl_df.tail(8)))

Top pos/neg correlated features against the target: 
                       corr w/ target
playMin_last5_mean              0.769
playMin_last10_mean             0.762
playMin_last5_median            0.756
playMin_last15_mean             0.754
playMin_last10_median           0.753
playMin_last20_mean             0.746
playMin_last15_median           0.744
playMin_last20_median           0.737
playMin_last1_mean              0.713
playPTS_last10_mean             0.665
playPTS_last15_mean             0.662
playPTS_last10_median           0.660
playPTS_last20_mean             0.658
playPTS_last5_mean              0.658
playPTS_last15_median           0.657
playPTS_last20_median           0.656
playStat                        0.651
playPTS_last5_median            0.644
playTO_last20_mean              0.557
playTO_last15_mean              0.556

                     corr w/ target
teamRslt_last5_mean          -0.001
teamLoc                      -0.002
teamLoc_last5_std            -0.002
team

In [32]:
# visualize the correlations
sort = list(correl_df.reset_index()['index'])
alt.Chart(correl_df.reset_index()).mark_line().encode(
    x=alt.X('index', sort=sort),
    y='corr w/ target:Q'
).properties(
    width=400,
    height=200
)

alt.Chart(...)

> * Sorry about the above plot. I can't set it to a reasonable size otherwise it won't render on GitHub.   
> * As expected, features like `playMin_last"n"` have very high correlation with the target. `playPTS` features also have high correlations, which is a bit surprising.  
> * We might consider enlarge "n" for `playMin_last"n"_std` features.  
> * Our base model to beat might be the `playMin_last_5_mean` feature.